In [1]:
!pip install opencv-python numpy tensorflow keras matplotlib


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Load Pretrained UNet Model for Segmentation
def load_unet_model(model_path="unet_model.h5"):
    return load_model(model_path)

# Apply Bilateral Filtering (Edge-Preserving)
def apply_bilateral_filter(image):
    return cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)

# Apply Adaptive Thresholding
def adaptive_thresholding(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    adaptive_thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                            cv2.THRESH_BINARY, 11, 2)
    return adaptive_thresh

# Edge-Aware Filtering (Guided Filter)
def guided_filter(image, guided):
    return cv2.ximgproc.guidedFilter(guided, image, radius=8, eps=50)

# Post-Processing: Remove Small Noise Regions (Connected Component Analysis)
def remove_small_regions(mask, min_size=500):
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    new_mask = np.zeros_like(mask)
    
    for i in range(1, num_labels):  # Ignore background
        if stats[i, cv2.CC_STAT_AREA] > min_size:
            new_mask[labels == i] = 255
            
    return new_mask

# Main Pipeline
def process_underwater_image(image_path, model_path="unet_model.h5"):
    # Load image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (256, 256))

    # Step 1: Noise Reduction
    filtered_image = apply_bilateral_filter(image)
    thresholded_image = adaptive_thresholding(filtered_image)

    # Step 2: Deep Learning-Based Segmentation
    model = load_unet_model(model_path)
    input_image = np.expand_dims(filtered_image / 255.0, axis=0)  # Normalize & expand dims
    segmented_mask = model.predict(input_image)[0, :, :, 0]  # Get segmentation output
    segmented_mask = (segmented_mask > 0.5).astype(np.uint8) * 255  # Threshold

    # Step 3: Post-processing
    refined_mask = guided_filter(segmented_mask, filtered_image)
    final_mask = remove_small_regions(refined_mask)

    # Display Results
    fig, axes = plt.subplots(1, 4, figsize=(12, 5))
    axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)); axes[0].set_title("Original")
    axes[1].imshow(thresholded_image, cmap="gray"); axes[1].set_title("Adaptive Thresholding")
    axes[2].imshow(segmented_mask, cmap="gray"); axes[2].set_title("UNet Segmentation")
    axes[3].imshow(final_mask, cmap="gray"); axes[3].set_title("Post-processed Mask")
    for ax in axes: ax.axis("off")
    plt.show()

# Example Usage (Run in a new cell)
process_underwater_image("sample_underwater.jpg", "unet_model.h5")


In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
import cv2
import numpy as np

# Update the path to match your repository structure
image_path = "path/to/your/image.jpg"  # Replace this with the correct path

# Read the noisy image
noisy_image = cv2.imread(image_path)

# Convert to grayscale if needed
gray_image = cv2.cvtColor(noisy_image, cv2.COLOR_BGR2GRAY)

# Apply Non-Local Means Denoising
denoised_image = cv2.fastNlMeansDenoising(gray_image, None, h=10, templateWindowSize=7, searchWindowSize=21)

# Save or display the denoised image
cv2.imwrite("denoised_image.jpg", denoised_image)
cv2.imshow("Denoised Image", denoised_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [2]:
import cv2
import os

# Set the correct path to your images folder (Update this)
input_folder = "C:/Users/USER/Documents/GitHub/Computer_Vision/Dataset/images"
output_folder = "C:/Users/USER/Documents/GitHub/Computer_Vision/Dataset/denoised_images"

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop through all images in the folder
for filename in os.listdir(input_folder):
    if filename.endswith((".png", ".jpg", ".jpeg")):  # Process only image files
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)

        if img is None:
            print(f"Skipping {filename}: Unable to read file")
            continue

        # Apply Non-Local Means Denoising
        denoised_img = cv2.fastNlMeansDenoisingColored(img, None, 10, 10, 7, 21)

        # Save the denoised image
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, denoised_img)

        print(f"Denoised and saved: {output_path}")

print("Processing complete!")


Denoised and saved: C:/Users/USER/Documents/GitHub/Computer_Vision/Dataset/denoised_images\d_r_47_.jpg
Denoised and saved: C:/Users/USER/Documents/GitHub/Computer_Vision/Dataset/denoised_images\d_r_598_.jpg
Denoised and saved: C:/Users/USER/Documents/GitHub/Computer_Vision/Dataset/denoised_images\f_r_1965_.jpg
Denoised and saved: C:/Users/USER/Documents/GitHub/Computer_Vision/Dataset/denoised_images\f_r_329_.jpg
Denoised and saved: C:/Users/USER/Documents/GitHub/Computer_Vision/Dataset/denoised_images\n_l_100_.jpg
Denoised and saved: C:/Users/USER/Documents/GitHub/Computer_Vision/Dataset/denoised_images\n_l_12_.jpg
Denoised and saved: C:/Users/USER/Documents/GitHub/Computer_Vision/Dataset/denoised_images\n_l_52_.jpg
Denoised and saved: C:/Users/USER/Documents/GitHub/Computer_Vision/Dataset/denoised_images\w_r_147_.jpg
Processing complete!
